# The ``Grid`` Object

Here we show how to instantiate a ``Grid`` object and use it to explore a grid file.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colormaps as cm
from matplotlib.colors import Normalize
from unyt import angstrom

The ``Grid`` object needs a file to load, these are HDF5 files that are available through the ``synthesizer-download`` command line tool (for more details see [the introduction to grids](grids.rst). By default, once downloaded these files are stored in the ``GRID_DIR`` directory. The default location of this directory is platform dependent, but the location can be found by import it and printing it.

In [ ]:
from synthesizer import GRID_DIR

print(GRID_DIR)

This directory can be overriden by setting the ``SYNTHESIZER_GRID_DIR`` environment variable.

Assuming the grid file is in the default location, all we need to do is pass the name of the grid we want to load to the ``Grid`` constructor. Note that the name of the grid can include the extension or not. If the extension is not included, it is assumed to be ``"hdf5"``. 

Here we will load the test grid (a simplified BPASS 2.2.1 grid).

In [ ]:
from synthesizer import Grid

grid = Grid("test_grid.hdf5")

If we are loading a grid from a diffrent location we can just pass that path to the ``grid_dir`` argument.

In [ ]:
import os
from shutil import copyfile

# Copy the grid file to the stated directory to
# demonstrate the grid_dir argument
copyfile(
    os.path.join(GRID_DIR, "test_grid.hdf5"),
    os.path.join("../../../tests/test_grid", "test_grid.hdf5"),
)

In [ ]:
grid = Grid("test_grid.hdf5", grid_dir="../../../tests/test_grid")

## Limiting the ``Grid``

A `Grid` can also take various arguments to limit the size of the grid, e.g. by isolating the `Grid` to a wavelength region of interest. This is particularly useful when making a large number of spectra from a high resolution `Grid`, where the memory footprint can become large.

### Passing a wavelength array

If you only care about a grid of specific wavelength values, you can pass this array and the `Grid` will automatically be interpolated onto the new wavelength array using [SpectRes](https://github.com/ACCarnall/SpectRes).

In [ ]:
# Define a new set of wavelengths
new_lams = np.logspace(2, 5, 1000) * angstrom

# Create a new grid
grid = Grid("test_grid", new_lam=new_lams)
print(grid.shape)

### Passing wavelength limits

If you don't want to modify the underlying grid resolution, but only care about a specific wavelength range, you can pass limits to truncate the grid at.

In [ ]:
# Create a new grid
grid = Grid("test_grid", lam_lims=(10**3 * angstrom, 10**4 * angstrom))
print(grid.shape)

## Plot a single grid point

We can plot the spectra at the location of a single point in our grid. First, we choose some age and metallicity.

In [ ]:
# Return to the unmodified grid
grid = Grid("test_grid")

log10age = 6.0  # log10(age/yr)
Z = 0.01  # metallicity

We then get the index location of that grid point for this age and metallicity

In [ ]:
grid_point = grid.get_grid_point(log10ages=log10age, metallicity=Z)

We can then loop over the available spectra (contained in `grid.spec_names`) and plot

In [ ]:
for spectra_type in grid.available_spectra:
    # Get `Sed` object
    sed = grid.get_sed_at_grid_point(grid_point, spectra_type=spectra_type)

    # Mask zero valued elements
    mask = sed.lnu > 0
    plt.plot(
        np.log10(sed.lam[mask]),
        np.log10(sed.lnu[mask]),
        lw=1,
        alpha=0.8,
        label=spectra_type,
    )

plt.legend(fontsize=8, labelspacing=0.0)
plt.xlim(2.3, 8)
plt.ylim(19, 25)
plt.xlabel(r"$\rm log_{10}(\lambda/\AA)$")
plt.ylabel(r"$\rm log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1} M_{\odot}^{-1})$")

## Plot ionising luminosities

We can also plot properties over the entire age and metallicity grid, such as the ionising luminosity. 

In the examples below we plot ionising luminosities for HI and HeII

In [ ]:
fig, ax = grid.plot_specific_ionising_lum(ion="HI")

In [ ]:
fig, ax = grid.plot_specific_ionising_lum(ion="HeII")

## Resampling `Grids`

If you want to resample a grid after instantiation, you can apply the `intrep_spectra` method:

In [ ]:
# Define a new set of wavelengths
new_lams = np.logspace(2, 5, 10000) * angstrom

print("The old grid had dimensions:", grid.spectra["incident"].shape)

# Get the grid interpolated onto the new wavelength array
grid.interp_spectra(new_lam=new_lams)

print("The interpolated grid has dimensions:", grid.spectra["incident"].shape)

Note that this will overwrite the spectra and wavelengths read from the file *in place*.
To get back to the original arrays, a separate `Grid` can be instantiated without the modified wavelength array.

## Collapsing `Grids`

While most of the models within synthesizer are capable of handling higher dimensionality grids (i.e. grids with more dimensions than `age` and `metallicity`), other workflows might require some method to reduce the dimensionality. 

This functionality is provided via the `collapse()` method, which collapses the grid over a specified axis.
There are three ways to actually collapse the grid, specified by the `method` keyword argument:

- `marginalize` over the entire axis. This is useful if you don't know anything about this parameter, and just want to adopt the average over it. You can specify the function used to marginalize with the keyword argument `marginalize_function`; the default is `np.average`. 
- Pick the value `nearest` to a specified value. If you know the value of the parameter you want to use, you can collapse the grid by picking the value closest to your specified value. For this, you need to specify the `value` keyword argument. 
- `interpolate` to a specified value. Similar to `nearest`, but with a linear interpolation to your specified `value`. This is useful in workflows where you can't adopt a discrete value, but be warned that interpolating over a coarse grid can give unrealistic results. You can apply a transformation to the axis before interpolating, e.g. to interpolate in log-space rather than linear space, with the keyword argument `pre_interp_function`. 

For example, here we collapse the grid over the metallicity axis:

In [ ]:
print("The old grid had dimensions:", grid.spectra["incident"].shape)
print("and axes:", ", ".join(grid.axes))

# Collapse the grid to a single metallicity value
grid.collapse("metallicities", value=0.03, method="nearest")

print("The collapsed grid has dimensions:", grid.spectra["incident"].shape)
print("and axes:", ", ".join(grid.axes))

Note that `collapse()` will overwrite the `Grid` _in place_. You can restore the grid to its original dimensionality by re-loading from the HDF5 file:

In [ ]:
# Re-load the original grid
grid = Grid("test_grid")

## Converting a `Grid` into an `Sed`

Any of the spectra arrays stored within a `Grid` can be returned as `Sed` objects (see the `Sed` [docs](../emissions/emission_objects/sed_example.ipynb)). This enables all of the analysis methods provide on an Sed to be used on the whole spectra grid. To do this we simply call `get_sed` with the spectra type we want to extract, and then use any of the included methods.

In [ ]:
# Get the sed object
sed = grid.get_sed(spectra_type="incident")

# Measure the balmer break for all spectra in the grid (limiting the output)
sed.measure_balmer_break()[5:10, 5:10]

## Working with flattened grids

Sometimes it's useful to work with flattened (i.e. one dimensional) versions of a grid of spectra, photometry etc. To facilitate this the `get_flattened_axes_values` method on `grid` can be used to get the flattened axes values.

In [ ]:
grid._axes_units["ages"]
grid._axes_values["ages"]

In [ ]:
# Get the balmer breaks of the entire grid
balmer_breaks = sed.measure_balmer_break()

# Get a flattened version of the Balmer break grid
flattened_balmer_breaks = balmer_breaks.flatten()

# Get the flattened version of the axes
flattened_axes_values = grid.get_flattened_axes_values()

# Normlise metallicities and create an array of colors
norm = Normalize(vmin=-4, vmax=-1.5, clip=True)
colors = cm["plasma"](norm(np.log10(flattened_axes_values["metallicities"])))

# Plot
plt.scatter(
    flattened_axes_values["ages"].to("Myr").value,
    flattened_balmer_breaks,
    c=colors,
)
plt.xscale("log")
plt.xlabel("ages/Myr")
plt.ylabel("Balmer break")
plt.show()

## Lines from `Grid` objects

Grids that have been post-processed through a photoionisation code (e.g. `Cloudy`) contain information on emission lines. We can see what lines are available on a grid by printing the ``available_lines`` attribute.

In [ ]:
print(grid.available_lines)

This is also reported if we give use the `print` function on a grid directly:

In [ ]:
print(grid)

## Extracting lines from a Grid

To demonstrate, we choose some age and metallicity and extract the spectra at that grid point. We can then get information on a single line, in this case H-$\beta$.

In [ ]:
log10age = 6.0  # log10(age/yr)
metallicity = 0.01

# find nearest grid point
grid_point = grid.get_grid_point(log10ages=log10age, metallicity=metallicity)
print(grid_point)
line = grid.get_lines(grid_point, "H 1 6562.80A")
print(line)

We can do this for a combination of lines (e.g. a doublet) if we just pass a comma-separated list of lines ids to the ``line_id`` argument.

In [ ]:
from synthesizer.emissions.utils import (
    Hb,
    O3b,
    O3r,
)

line = grid.get_lines(grid_point, ", ".join([Hb, O3r, O3b]))
print(line)

We can also get a collection of individual lines by passing a list of line ids to the ``line_id`` argument. 

In [ ]:
line = grid.get_lines(grid_point, [Hb, O3r, O3b])
print(line)

If we don't pass a list then it defaults to returning all available lines.


In [ ]:
lines = grid.get_lines(grid_point)
print(lines)

## Ratios as a function of metallicity

To show the dependence on stellar metallicity we can loop over the metallicity grid:

In [ ]:
ratio_id = "R23"
ia = 0  # 1 Myr old for test grid
ratios = []
for iZ, Z in enumerate(grid.metallicity):
    grid_point = (ia, iZ)
    lines = grid.get_lines(grid_point)
    ratios.append(lines.get_ratio(ratio_id))

Zsun = grid.metallicity / 0.0124
plt.plot(Zsun, ratios)
plt.xlim([0.01, 1])
plt.ylim([1, 20])
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$Z/Z_{\odot}$")
plt.ylabel(rf"{ratio_id}")
plt.show()

## Line Diagrams as a function of metallicity

We can also generate diagrams using pairs of line ratios, such as the famous Baldwin, Phillips & Terlevich (BPT) diagram.

``line_ratios`` also contains some classification regions (e.g. [Kewley+13](https://ui.adsabs.harvard.edu/abs/2013ApJ...774L..10K/abstract) and [Kauffmann+03](https://ui.adsabs.harvard.edu/abs/2003MNRAS.346.1055K/abstract)) that we can plot:

In [ ]:
from synthesizer.emissions import line_ratios

diagram_id = "BPT-NII"
ia = 0  # 1 Myr old for test grid
x = []
y = []
for iZ, Z in enumerate(grid.metallicity):
    grid_point = (ia, iZ)
    lines = grid.get_lines(grid_point)
    x_, y_ = lines.get_diagram(diagram_id)
    x.append(x_)
    y.append(y_)


# Plot the Kewley SF/AGN dividing line
fig, ax = plt.subplots()
ax.plot(x, y)
logNII_Ha = np.arange(-2.0, 1.0, 0.01)
logOIII_Hb = line_ratios.plot_bpt_kewley01(
    logNII_Ha, fig=fig, ax=ax, show=True, c="k", lw="2", alpha=0.3
)